In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [4]:
X = df.iloc[:,:-1]
y =  df.iloc[:,-1]

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [6]:
X = scaler.fit_transform(X)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [73]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout

In [9]:
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = 8))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

C:\Users\LAPTOP\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_data = (X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.6417 - loss: 0.6475 - val_accuracy: 0.7468 - val_loss: 0.5961
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7020 - loss: 0.6014 - val_accuracy: 0.7792 - val_loss: 0.5612
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7134 - loss: 0.5690 - val_accuracy: 0.7857 - val_loss: 0.5388
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7296 - loss: 0.5462 - val_accuracy: 0.7922 - val_loss: 0.5216
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7313 - loss: 0.5286 - val_accuracy: 0.7922 - val_loss: 0.5082
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7459 - loss: 0.5145 - val_accuracy: 0.7922 - val_loss: 0.4978
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7524 - loss: 0.5038 - val_accuracy: 0.8052 - val_loss: 0.4921
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7590 - loss: 0.4948 - val_accuracy: 0.7

In [16]:
import keras_tuner as kt

In [17]:
def build_model(hp):
    model = Sequential()

    model.add(Dense(32, activation = 'relu', input_dim= 8))
    model.add(Dense(1,activation = 'sigmoid'))

    optimizer = hp.Choice('optimizer', values = ['adam', 'rmsprop', 'adadelta', 'sgd'])
    model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [21]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5,overwrite=True)

C:\Users\LAPTOP\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
tuner.search(X_train, y_train, epochs =5, validation_data=(X_test, y_test), verbose=1)

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.7727272510528564

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 15s


In [23]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [24]:
model = tuner.get_best_models(num_models=1)[0]

C:\Users\LAPTOP\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\LAPTOP\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [25]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
model.fit(X_train, y_train, epochs = 100, initial_epoch = 6, validation_data = (X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.7606 - loss: 0.5160 - val_accuracy: 0.7662 - val_loss: 0.5001
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7557 - loss: 0.5017 - val_accuracy: 0.7662 - val_loss: 0.4898
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7557 - loss: 0.4923 - val_accuracy: 0.7662 - val_loss: 0.4826
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7622 - loss: 0.4855 - val_accuracy: 0.7727 - val_loss: 0.4778
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7638 - loss: 0.4797 - val_accuracy: 0.7727 - val_loss: 0.4729
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7720 - loss: 0.4742 - val_accuracy: 0.7727 - val_loss: 0.4697
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7736 - loss: 0.4701 - val_accuracy: 0.7727 - val_loss: 0.4667
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7720 - loss: 0.4662 - val_accuracy:

In [49]:
def build_model(hp):

    model = Sequential()

    units = hp.Int('units', min_value = 8,max_value = 128)

    model.add(Dense(units = units, activation = 'relu', input_dim = 8))
    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

    return model

In [50]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials = 10, project_name = 'srk')

C:\Users\LAPTOP\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [51]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test), verbose=1)

Trial 10 Complete [00h 00m 03s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 33s


In [52]:
tuner.get_best_hyperparameters()[0].values

{'units': 85}

In [53]:
model.fit(X_train, y_train, epochs=100, initial_epoch = 6, validation_data = (X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8371 - loss: 0.3775 - val_accuracy: 0.8117 - val_loss: 0.4828
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8339 - loss: 0.3777 - val_accuracy: 0.8052 - val_loss: 0.4827
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8355 - loss: 0.3772 - val_accuracy: 0.8182 - val_loss: 0.4818
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8339 - loss: 0.3770 - val_accuracy: 0.8182 - val_loss: 0.4831
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8322 - loss: 0.3773 - val_accuracy: 0.8052 - val_loss: 0.4821
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8355 - loss: 0.3767 - val_accuracy: 0.7922 - val_loss: 0.4840
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8322 - loss: 0.3764 - val_accuracy: 0.8052 - val_loss: 0.4816
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8371 - loss: 0.3761 - val_accuracy:

In [56]:
def build_model(hp):

    model = Sequential()

    model.add(Dense(85, activation = 'relu', input_dim = 8))

    for i in range(hp.Int('num_layers', min_value=1, max_value = 10)):
        model.add(Dense(85,activation = 'relu'))
    
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

    return model

In [57]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy',
                        max_trials =3,
                        directory = 'my_dir',
                        project_name = 'sskd')

C:\Users\LAPTOP\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [58]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test), verbose =1)

Trial 3 Complete [00h 00m 06s]
val_accuracy: 0.8246753215789795

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 15s


In [59]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 9}

In [60]:
model = tuner.get_best_models(num_models = 1)[0]

C:\Users\LAPTOP\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\LAPTOP\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 24 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [61]:
model.fit(X_train, y_train, epochs = 100, initial_epoch = 6, validation_data = (X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - accuracy: 0.7769 - loss: 0.4624 - val_accuracy: 0.7857 - val_loss: 0.4826
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7997 - loss: 0.4278 - val_accuracy: 0.7922 - val_loss: 0.4855
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7997 - loss: 0.4308 - val_accuracy: 0.7662 - val_loss: 0.4947
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8094 - loss: 0.4137 - val_accuracy: 0.8117 - val_loss: 0.4944
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8127 - loss: 0.4094 - val_accuracy: 0.7922 - val_loss: 0.4844
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8290 - loss: 0.3903 - val_accuracy: 0.7922 - val_loss: 0.4893
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8420 - loss: 0.3742 - val_accuracy: 0.7987 - val_loss: 0.5285
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8453 - loss: 0.3552 - val_accurac

In [76]:
def build_model(hp):
    model = Sequential()

    counter = 0

    for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):

        if counter == 0:

            model.add(Dense(
                hp.Int('units' + str(i), min_value = 8, max_value = 128, step=8),
                activation = hp.Choice('activation'+str(i), values = ['sigmoid','relu', 'tanh']),
                input_dim = 8))
            model.add(Dropout(hp.Choice('Dropout'+str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

        else:
            model.add(Dense(
                hp.Int('units' + str(i), min_value = 8, max_value = 128, step=8),
                activation = hp.Choice('activation'+str(i), values = ['sigmoid','relu', 'tanh'])))
            model.add(Dropout(hp.Choice('Dropout'+str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter += 1

    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer = hp.Choice('optimizer', values = ['rmsprop', 'adam', 'sgd', 'nadam', 'adadelta']),
                  loss = 'binary_crossentropy', 
                  metrics=['accuracy'])
    
    return model

            
                      
        

In [77]:
tuner = kt.RandomSearch(build_model,objective = 'val_accuracy', max_trials = 5, directory = 'my_dir',project_name = 'final2')

C:\Users\LAPTOP\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [78]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test,y_test))

Trial 5 Complete [00h 00m 08s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.7922077775001526
Total elapsed time: 00h 00m 36s


In [79]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 104,
 'activation0': 'tanh',
 'Dropout0': 0.2,
 'optimizer': 'adam',
 'units1': 48,
 'activation1': 'tanh',
 'Dropout1': 0.7,
 'units2': 96,
 'activation2': 'relu',
 'Dropout2': 0.5,
 'units3': 128,
 'activation3': 'sigmoid',
 'Dropout3': 0.6,
 'units4': 16,
 'activation4': 'tanh',
 'Dropout4': 0.8,
 'units5': 48,
 'activation5': 'sigmoid',
 'Dropout5': 0.7}

In [80]:
model = tuner.get_best_models(num_models=1)[0]

C:\Users\LAPTOP\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\LAPTOP\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [81]:
model.fit(X_train,y_train,epochs =200, initial_epoch = 5, validation_data = (X_test, y_test))

Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.7508 - loss: 0.5113 - val_accuracy: 0.7792 - val_loss: 0.4845
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7622 - loss: 0.4862 - val_accuracy: 0.7857 - val_loss: 0.4701
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7785 - loss: 0.4799 - val_accuracy: 0.7857 - val_loss: 0.4651
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7671 - loss: 0.4821 - val_accuracy: 0.7857 - val_loss: 0.4641
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7720 - loss: 0.4672 - val_accuracy: 0.7857 - val_loss: 0.4651
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7720 - loss: 0.4668 - val_accuracy: 0.7792 - val_loss: 0.4674
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7622 - loss: 0.4696 - val_accuracy: 0.7857 - val_loss: 0.4659
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7720 - loss: 0.4696 - val_accuracy